This notebook is for a neural network (NN) on curves with size of Sha equal to 4 and 9 and removing one BSD feature at a time. This includes both the original and log-transformed data.

In [1]:
from lib import utils, models, executor
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

# fix the seed for reproducibility
seed = 42

# 1. Create balanced dataset of elliptic curves with size of the Tate-Shafarevich group equal to 4 and 9 containing all BSD features

In [2]:
# load your data here. The following ensure this will work on Windows as well as Unix
path = Path("..") / "data_files" / "sha"/ "ecq_sha_B_100_conds_1_500000_reg.parquet"
df = utils.load_data(path)

Loaded the dataset with 120 features and 3064705 curves..


In [3]:
len_9 = df[df['sha'] == 9].shape[0]
df_balanced = df[df['sha'] == 4].sample(len_9, random_state=seed) 
df_balanced = pd.concat([df_balanced, df[df['sha'] == 9]])
df_balanced.sha.value_counts()

sha
4    50428
9    50428
Name: count, dtype: int64

In [4]:
#Get columns with all the BSD features, from which we will eventually remove one at a time
bsd_features = ['special_value', 'torsion', 'real_period', 'regulator', 'tamagawa_product', 'sha']

df_balanced_bsd = df_balanced[bsd_features].copy()

In [5]:
df_balanced_bsd.head(5)

,special_value,torsion,real_period,regulator,tamagawa_product,sha
334625,2.19751,2,0.54938,1.0,4,4
1086182,3.22805,1,0.80701,1.0,1,4
1782926,3.98612,2,0.49826,1.0,8,4
2484030,2.99537,1,0.09361,1.0,8,4
3053287,2.23394,1,0.09308,1.0,6,4


# 2. Delete one feature at a time on original data

The best accuracy is about 97% when removing either the size of torsion or the regulator.

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
# Neural Net classifier

# choose model parameters
hidden_units = [128,64,32]

# check if we have cuda available
device = 'cuda'

# Map labels to range starting from 0
df_BSD_label_mapped = df_balanced_bsd.copy()

# Neural net training for binary classification expects labels to start at 0
df_BSD_label_mapped['sha'] = df_BSD_label_mapped['sha'].map({4: 0, 9: 1})

# choose training parameters
loss_func = nn.CrossEntropyLoss()
num_epochs = 100
lr = 0.001
evaluator = accuracy_score

# Initialize an empty DataFrame to store the results
results_df_nn = pd.DataFrame(columns=['Feature Deleted', 'Accuracy'])

for i in range(len(bsd_features)-1):
    print(f'Running model without {bsd_features[i]}..')
    df_sub = df_BSD_label_mapped.drop(columns=[bsd_features[i]])
    input_dim, output_dim = utils.get_input_output_dim(df_sub, 'sha', if_regression=False)
    model = models.VanillaNN(input_dim, hidden_units, output_dim, if_dropout=False, dropout_rate=0.3, if_batchnorm=True).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr) # reinitialize optimizer
    train_dataloader, val_dataset, test_dataset = utils.prepare_data(df_sub, 'sha', device, if_regression=False, random_state=seed)
    model, train_eval_hist, val_eval_hist, train_loss_hist, val_loss_hist = executor.train(model, train_dataloader, val_dataset, loss_func, evaluator, optimizer, num_epochs, if_regression=False, verbose=False)
    acc = executor.test(model, test_dataset, evaluator, if_regression=False)
    print(f"Test accuracy: {acc:0.3f}")
    print('----------------------------------')
    
   # Append the results to the DataFrame
    results_df_nn = pd.concat([results_df_nn, pd.DataFrame([{'Feature Deleted': bsd_features[i], 'Accuracy': acc}])], ignore_index=True)

Running model without special_value..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 89 with Training CrossEntropyLoss : 0.5473, Validation CrossEntropyLoss : 0.5517. Training accuracy_score : 0.7100, Validation accuracy_score : 0.7076, to ../trained_models/model.pth.
Test accuracy: 0.705
----------------------------------
Running model without torsion..
The input dimension is 4 and the output dimension is 2.


/tmp/ipykernel_1103/125803182.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_nn = pd.concat([results_df_nn, pd.DataFrame([{'Feature Deleted': bsd_features[i], 'Accuracy': acc}])], ignore_index=True)


Save the model from epoch 80 with Training CrossEntropyLoss : 0.0953, Validation CrossEntropyLoss : 0.0891. Training accuracy_score : 0.9658, Validation accuracy_score : 0.9655, to ../trained_models/model.pth.
Test accuracy: 0.966
----------------------------------
Running model without real_period..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 96 with Training CrossEntropyLoss : 0.6349, Validation CrossEntropyLoss : 0.6340. Training accuracy_score : 0.6262, Validation accuracy_score : 0.6269, to ../trained_models/model.pth.
Test accuracy: 0.635
----------------------------------
Running model without regulator..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 96 with Training CrossEntropyLoss : 0.0668, Validation CrossEntropyLoss : 0.0633. Training accuracy_score : 0.9677, Validation accuracy_score : 0.9675, to ../trained_models/model.pth.
Test accuracy: 0.967
----------------------------------
Running model without tamagawa_product..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 89 with Training CrossEntropyLoss : 0.2811, Validation CrossEntropyLoss : 0.2779. Training accuracy_score : 0.8279, Validation accuracy_score : 0.8297, to ../trained_models/model.pth.
Test accuracy: 0.826
----------------------------------


In [8]:
print(results_df_nn)

    Feature Deleted  Accuracy
0     special_value  0.704739
1           torsion  0.966389
2       real_period  0.635237
3         regulator  0.967034
4  tamagawa_product  0.825550


# 3. Delete one feature at a time on log-transformed data
Accuracy is almost at 99% when removing either the size of torsion or the regulator.

In [10]:
# Neural Net classifier on log-transformed data

# choose model parameters
hidden_units = [128,64,32]

# check if we have cuda available
device = 'cuda'

# Map labels to range starting from 0
df_BSD_label_mapped = df_balanced_bsd.copy()
for col in bsd_features[:-1]:
    df_BSD_label_mapped[col]=df_BSD_label_mapped[col].apply(np.log)

# Neural net training for binary classification expects labels to start at 0
df_BSD_label_mapped['sha'] = df_BSD_label_mapped['sha'].map({4: 0, 9: 1})

# choose training parameters
loss_func = nn.CrossEntropyLoss()
num_epochs = 100
lr = 0.001
evaluator = accuracy_score

# Initialize an empty DataFrame to store the results
results_df_nn_log = pd.DataFrame(columns=['Feature Deleted', 'Accuracy'])

for i in range(len(bsd_features)-1):
    print(f'Running model without {bsd_features[i]}..')
    df_sub = df_BSD_label_mapped.drop(columns=[bsd_features[i]])
    input_dim, output_dim = utils.get_input_output_dim(df_sub, 'sha', if_regression=False)
    model = models.VanillaNN(input_dim, hidden_units, output_dim, if_dropout=False, dropout_rate=0.3, if_batchnorm=True).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr) # reinitialize optimizer
    train_dataloader, val_dataset, test_dataset = utils.prepare_data(df_sub, 'sha', device, if_regression=False, random_state=seed)
    model, train_eval_hist, val_eval_hist, train_loss_hist, val_loss_hist = executor.train(model, train_dataloader, val_dataset, loss_func, evaluator, optimizer, num_epochs, if_regression=False, verbose=False)
    acc = executor.test(model, test_dataset, evaluator, if_regression=False)
    print(f"Test accuracy: {acc:0.3f}")
    print('----------------------------------')
    
   # Append the results to the DataFrame
    results_df_nn_log = pd.concat([results_df_nn_log, pd.DataFrame([{'Feature Deleted': bsd_features[i], 'Accuracy': acc}])], ignore_index=True)

Running model without special_value..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 64 with Training CrossEntropyLoss : 0.5403, Validation CrossEntropyLoss : 0.5451. Training accuracy_score : 0.7125, Validation accuracy_score : 0.7089, to ../trained_models/model.pth.
Test accuracy: 0.709
----------------------------------
Running model without torsion..
The input dimension is 4 and the output dimension is 2.


/tmp/ipykernel_1103/14240141.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df_nn_log = pd.concat([results_df_nn_log, pd.DataFrame([{'Feature Deleted': bsd_features[i], 'Accuracy': acc}])], ignore_index=True)


Save the model from epoch 8 with Training CrossEntropyLoss : 0.0449, Validation CrossEntropyLoss : 0.0406. Training accuracy_score : 0.9886, Validation accuracy_score : 0.9898, to ../trained_models/model.pth.
Test accuracy: 0.990
----------------------------------
Running model without real_period..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 57 with Training CrossEntropyLoss : 0.6333, Validation CrossEntropyLoss : 0.6325. Training accuracy_score : 0.6320, Validation accuracy_score : 0.6283, to ../trained_models/model.pth.
Test accuracy: 0.637
----------------------------------
Running model without regulator..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 99 with Training CrossEntropyLoss : 0.0288, Validation CrossEntropyLoss : 0.0270. Training accuracy_score : 0.9895, Validation accuracy_score : 0.9904, to ../trained_models/model.pth.
Test accuracy: 0.988
----------------------------------
Running model without tamagawa_product..
The input dimension is 4 and the output dimension is 2.


Save the model from epoch 98 with Training CrossEntropyLoss : 0.2418, Validation CrossEntropyLoss : 0.2171. Training accuracy_score : 0.9121, Validation accuracy_score : 0.9129, to ../trained_models/model.pth.
Test accuracy: 0.911
----------------------------------


In [11]:
print(results_df_nn_log)

    Feature Deleted  Accuracy
0     special_value  0.708804
1           torsion  0.989639
2       real_period  0.636823
3         regulator  0.987904
4  tamagawa_product  0.910817
